In [ ]:
!pip3 install tweepy datetime

In [ ]:
!python3 get_tweets.py

In [2]:
import json

In [7]:
# Abrir arquivo e ler linhas
with open('./collected_tweets_2021-03-09-22-16-43.txt', 'r') as read_file:
    tweets = read_file.readlines()

In [8]:
print(len(tweets))
print(type(tweets))

5046
<class 'list'>


In [13]:
# Parseia o json de tweet e exporta em arquiv .json
with open('tweet.json', 'w') as json_file:
    json.dump(
        json.loads(json.loads(tweets[0])),
        json_file
    )

In [10]:
type(json.loads(tweets[0]))

str

In [11]:
# Parseia o json 2x. Transformando a string json para um dicionario
json.loads(
    json.loads(tweets[0])
)

{'created_at': 'Wed Mar 10 01:16:39 +0000 2021',
 'id': 1369457138284781568,
 'id_str': '1369457138284781568',
 'text': '@GOPLeader And the Republicans are dumb as a rock politically. They allowed #biden to steal the election by fraud,… https://t.co/6sKjMRkYdo',
 'display_text_range': [11, 140],
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': True,
 'in_reply_to_status_id': 1369299591489806345,
 'in_reply_to_status_id_str': '1369299591489806345',
 'in_reply_to_user_id': 19739126,
 'in_reply_to_user_id_str': '19739126',
 'in_reply_to_screen_name': 'GOPLeader',
 'user': {'id': 354865786,
  'id_str': '354865786',
  'name': 'Congressgohome',
  'screen_name': 'Congressgohome',
  'location': 'Winchester, VA',
  'url': None,
  'description': 'Congress should move most of its sessions to the Internet, stop spending so much time in DC.  Likes and Re-Tweets do not imply any endorsement.',
  'translator_type': 'none',
  'protected':

In [12]:
type(
    json.loads(
    json.loads(tweets[0])
    )
)

dict